In [ ]:
import string
from nltk.corpus import wordnet as wn
import random

specialty_to_subspecialties = {
    "Cardiology": ["Interventional Cardiology", "Electrophysiology", "Heart Failure", "Pediatric Cardiology", "Cardiac Imaging"],
    "Dermatology": ["Pediatric Dermatology", "Cosmetic Dermatology", "Dermatopathology", "Mohs Surgery"],
    "Emergency Medicine": ["Toxicology", "Ultrasound", "Disaster Medicine", "Pediatric Emergency Medicine"],
    "Endocrinology": ["Diabetes", "Thyroid Disorders", "Adrenal Disorders", "Pituitary Disorders"],
    "Family Medicine": ["Preventive Care", "Geriatric Medicine", "Adolescent Medicine", "Women's Health"],
    "Gastroenterology": ["Hepatology", "Inflammatory Bowel Disease", "Pancreatology", "Advanced Endoscopy"],
    "General Surgery": ["Trauma Surgery", "Minimally Invasive Surgery", "Breast Surgery", "Colorectal Surgery"],
    "Geriatrics": ["Memory Disorders", "Palliative Care", "Multimorbidity", "Falls and Balance"],
    "Hematology": ["Leukemia", "Lymphoma", "Coagulation Disorders", "Anemia"],
    "Infectious Diseases": ["HIV/AIDS", "Tropical Diseases", "Antimicrobial Stewardship", "Tuberculosis"],
    "Internal Medicine": ["Hospital Medicine", "Primary Care", "Clinical Pharmacology", "Medical Oncology"],
    "Nephrology": ["Chronic Kidney Disease", "Dialysis", "Transplant Nephrology", "Glomerular Diseases"],
    "Neurology": ["Stroke", "Epilepsy", "Multiple Sclerosis", "Neuromuscular Disorders"],
    "Obstetrics and Gynecology": ["Maternal-Fetal Medicine", "Gynecologic Oncology", "Reproductive Endocrinology", "Urogynecology"],
    "Oncology": ["Breast Cancer", "Lung Cancer", "Palliative Oncology", "Hematologic Malignancies"],
    "Ophthalmology": ["Retina", "Glaucoma", "Cornea", "Pediatric Ophthalmology"],
    "Orthopedics": ["Sports Medicine", "Joint Replacement", "Pediatric Orthopedics", "Spine Surgery"],
    "Otolaryngology": ["Rhinology", "Laryngology", "Otology", "Head and Neck Surgery"],
    "Pediatrics": ["Pediatric Cardiology", "Pediatric Neurology", "Pediatric Endocrinology", "Pediatric Infectious Diseases"],
    "Plastic Surgery": ["Reconstructive Surgery", "Aesthetic Surgery", "Burn Surgery", "Craniofacial Surgery"],
    "Psychiatry": ["Child and Adolescent Psychiatry", "Addiction Psychiatry", "Forensic Psychiatry", "Geriatric Psychiatry"],
    "Pulmonology": ["Sleep Medicine", "Critical Care", "Interstitial Lung Disease", "Pulmonary Hypertension"],
    "Radiology": ["Neuroradiology", "Interventional Radiology", "Musculoskeletal Radiology", "Breast Imaging"],
    "Rheumatology": ["Lupus", "Rheumatoid Arthritis", "Vasculitis", "Spondyloarthritis"],
    "Urology": ["Pediatric Urology", "Urologic Oncology", "Female Urology", "Endourology"]
}

subspecialty_keywords = {
    "Interventional Cardiology": ["stent", "angioplasty", "catheter", "coronary", "artery", "blockage", "ischemia"],
    "Electrophysiology": ["arrhythmia", "palpitations", "pacemaker", "ECG", "irregular heartbeat", "tachycardia"],
    "Heart Failure": ["shortness of breath", "fatigue", "fluid retention", "swollen ankles", "CHF", "weak heart"],
    "Pediatric Cardiology": ["congenital heart", "blue baby", "heart murmur", "child", "pediatric", "cyanosis"],
    "Cardiac Imaging": ["echocardiogram", "MRI heart", "CT angiogram", "cardiac ultrasound"],

    "Pediatric Dermatology": ["eczema", "rashes", "chickenpox", "diaper rash", "child", "skin bumps"],
    "Cosmetic Dermatology": ["botox", "fillers", "laser", "acne scars", "wrinkle treatment"],
    "Dermatopathology": ["skin biopsy", "melanoma", "skin cancer", "basal cell", "histopathology"],
    "Mohs Surgery": ["skin cancer removal", "precise excision", "basal cell", "surgical margins"],

    "Toxicology": ["overdose", "poisoning", "toxins", "drug ingestion", "chemical exposure", "antidote"],
    "Ultrasound": ["FAST scan", "ultrasound trauma", "internal bleeding", "POCUS"],
    "Disaster Medicine": ["earthquake", "mass casualty", "triage", "injury", "disaster response"],
    "Pediatric Emergency Medicine": ["fever in infant", "seizure", "trauma child", "difficulty breathing"],

    "Diabetes": ["high blood sugar", "insulin", "glucose", "polyuria", "polydipsia", "HbA1c"],
    "Thyroid Disorders": ["hypothyroidism", "hyperthyroidism", "goiter", "TSH", "fatigue", "weight change"],
    "Adrenal Disorders": ["Addison’s", "Cushing's", "cortisol", "electrolyte imbalance"],
    "Pituitary Disorders": ["growth hormone", "acromegaly", "prolactin", "pituitary tumor"],

    "Hospital Medicine": ["inpatient", "hospitalized", "rounds", "ward", "discharge", "admission"],
    "Primary Care": ["check-up", "follow-up", "routine care", "health maintenance", "chronic disease"],
    "Clinical Pharmacology": ["drug interaction", "dose adjustment", "side effects", "pharmacokinetics"],
    "Medical Oncology": ["chemotherapy", "cancer treatment", "tumor", "radiation", "metastasis"],

    "Chronic Kidney Disease": ["creatinine", "proteinuria", "dialysis", "kidney function", "nephron loss"],
    "Dialysis": ["hemodialysis", "peritoneal dialysis", "access site", "fluid removal", "ESRD"],
    "Transplant Nephrology": ["kidney transplant", "rejection", "immunosuppression", "donor", "recipient"],
    "Glomerular Diseases": ["proteinuria", "nephrotic syndrome", "glomerulonephritis", "edema", "urinalysis"],

    "Stroke": ["sudden weakness", "slurred speech", "drooping face", "CT head", "clot", "TIA"],
    "Epilepsy": ["seizure", "convulsion", "aura", "EEG", "anti-epileptic"],
    "Multiple Sclerosis": ["vision loss", "tingling", "numbness", "relapsing", "neurological"],
    "Neuromuscular Disorders": ["muscle weakness", "ALS", "myasthenia", "EMG", "fatigue"],

    "Maternal-Fetal Medicine": ["high-risk pregnancy", "gestational diabetes", "ultrasound", "preterm labor"],
    "Gynecologic Oncology": ["ovarian cancer", "uterine cancer", "hysterectomy", "tumor markers"],
    "Reproductive Endocrinology": ["infertility", "IVF", "hormonal imbalance", "menstrual cycle"],
    "Urogynecology": ["incontinence", "pelvic floor", "prolapse", "bladder dysfunction"],

    "Breast Cancer": ["lump", "mammogram", "biopsy", "HER2", "hormone receptor"],
    "Lung Cancer": ["cough", "chest pain", "smoking", "mass", "bronchoscopy"],
    "Palliative Oncology": ["pain relief", "terminal", "end-of-life", "supportive care"],
    "Hematologic Malignancies": ["leukemia", "lymphoma", "bone marrow", "chemotherapy"],

    "Retina": ["detached retina", "floaters", "macular degeneration", "vision loss"],
    "Glaucoma": ["eye pressure", "optic nerve", "tunnel vision", "eye drops"],
    "Cornea": ["transplant", "abrasion", "infection", "keratitis", "contact lens"],
    "Pediatric Ophthalmology": ["lazy eye", "strabismus", "blocked tear duct", "child vision"],

    "Sports Medicine": ["injury", "sprain", "ACL tear", "rehab", "athlete"],
    "Joint Replacement": ["hip replacement", "knee surgery", "prosthesis", "arthritis"],
    "Pediatric Orthopedics": ["limp", "fracture", "scoliosis", "growth plate", "clubfoot"],
    "Spine Surgery": ["herniated disc", "spinal fusion", "sciatica", "stenosis"],

    "Rhinology": ["sinus", "nasal congestion", "polyps", "smell loss", "deviated septum"],
    "Laryngology": ["voice change", "hoarseness", "vocal cord", "throat pain"],
    "Otology": ["ear infection", "hearing loss", "tinnitus", "vertigo", "ear discharge"],
    "Head and Neck Surgery": ["thyroid mass", "neck tumor", "lymph node", "ENT cancer"],

    "Pediatric Neurology": ["seizures in child", "development delay", "autism", "headache", "motor skills"],
    "Pediatric Endocrinology": ["growth delay", "early puberty", "diabetes child", "thyroid child"],
    "Pediatric Infectious Diseases": ["fever", "rash", "infections", "vaccines", "antibiotics child"],

    "Reconstructive Surgery": ["scar revision", "trauma repair", "flap surgery", "skin graft"],
    "Aesthetic Surgery": ["cosmetic", "rhinoplasty", "liposuction", "facelift"],
    "Burn Surgery": ["burns", "grafting", "wound care", "fluid resuscitation"],
    "Craniofacial Surgery": ["cleft palate", "skull deformity", "pediatric facial surgery"],

    "Child and Adolescent Psychiatry": ["ADHD", "anxiety", "depression", "autism", "behavioral issues"],
    "Addiction Psychiatry": ["substance abuse", "detox", "rehab", "opioid", "alcohol"],
    "Forensic Psychiatry": ["legal", "competency", "criminal", "mental illness court"],
    "Geriatric Psychiatry": ["memory loss", "agitation", "dementia", "elderly mental health"],

    "Sleep Medicine": ["apnea", "snoring", "CPAP", "insomnia", "daytime fatigue"],
    "Critical Care": ["ICU", "ventilator", "sepsis", "shock", "intubation"],
    "Interstitial Lung Disease": ["fibrosis", "dyspnea", "cough", "CT chest"],
    "Pulmonary Hypertension": ["shortness of breath", "PAH", "RV strain", "heart-lung"],

    "Neuroradiology": ["MRI brain", "stroke imaging", "CT head", "white matter", "contrast"],
    "Interventional Radiology": ["catheter", "embolization", "biopsy", "minimally invasive"],
    "Musculoskeletal Radiology": ["bone scan", "MRI joint", "arthritis imaging", "sports injury"],
    "Breast Imaging": ["mammogram", "ultrasound breast", "BI-RADS", "MRI breast"],

    "Lupus": ["rash", "autoimmune", "ANA positive", "joint pain", "fatigue"],
    "Rheumatoid Arthritis": ["joint stiffness", "morning stiffness", "RF", "anti-CCP"],
    "Vasculitis": ["inflammation vessels", "rash", "fever", "ANCA", "organ damage"],
    "Spondyloarthritis": ["back pain", "HLA-B27", "AS", "enthesitis", "uveitis"],

    "Pediatric Urology": ["bedwetting", "undescended testicle", "UTI child", "phimosis"],
    "Urologic Oncology": ["prostate cancer", "bladder tumor", "kidney mass", "hematuria"],
    "Female Urology": ["incontinence", "bladder control", "pelvic organ prolapse"],
    "Endourology": ["kidney stones", "laser lithotripsy", "ureteroscopy", "stent"],

}

stopwords = ['in','what','who','is','a','at','is', 'the', 'an', 'and', 'to', 'from', 'they', 'is', 'are', 'for', 'be', 'with', 'of', 'has', 'have']

for key in subspecialty_keywords.keys():
    temp_list =[]
    for keyword in subspecialty_keywords[key]:
        split_keyword = keyword.split(' ')
        for k in split_keyword:
            if(not k in stopwords):
                temp_list.append(k)
    subspecialty_keywords[key]=temp_list

def subspecialty_relevance_avg(subspecialty, keyword):
    n_subspecialty_keywords = len(subspecialty_keywords[subspecialty])
    score_sum = 0
    for ssk in subspecialty_keywords[subspecialty]:
        try:
            score_sum += wn.synsets(ssk)[0].path_similarity(wn.synsets(keyword)[0])
        except:
            n_subspecialty_keywords-=1
            continue
    if(n_subspecialty_keywords!=0):
        normalized_score = score_sum/n_subspecialty_keywords
        return normalized_score
    else:
        return 0
    
def subspecialty_relevance_max(subspecialty, keyword):
    score = 0
    for ssk in subspecialty_keywords[subspecialty]:
        try:
            new_score = wn.synsets(ssk)[0].path_similarity(wn.synsets(keyword)[0])
            if(new_score>score):
                score = new_score
        except:
            continue
        return score

print(subspecialty_relevance_avg("Lupus", "rash"))

for key in subspecialty_keywords.keys():
    score = subspecialty_relevance_avg(key,"rash")
    if(score==0):
        print(key)

0.23450061842918982


In [ ]:
import string
from nltk.corpus import wordnet as wn

patient_descriptions = ["The patient reports symptoms including chest pain, artery blockage, and ischemia. They are seeking help for what seems to be an interventional cardiology issue.",
                        "The patient is a child experiencing skin bumps, chickenpox, and diaper rash. They are seeking help for what seems to be a pediatric dermatology issue.",
                        "The patient suddenly developed slurred speech, facial drooping, and arm weakness. They are seeking help for what seems to be a stroke-related issue.",
                        "The patient is experiencing joint stiffness, morning stiffness, and has a history of positive RF results. They are seeking help for what seems to be a rheumatoid arthritis issue.",
                        "A child is presenting with recurring UTIs, bedwetting, and signs of phimosis. They are seeking help for what seems to be a pediatric urology issue.",
                        "The patient reports irregular bleeding, pelvic mass, and a family history of ovarian cancer. They are seeking help for what seems to be a gynecologic oncology issue.",
                        "The patient has been having seizures, reports auras before episodes, and is currently on anti-epileptic medication. They are seeking help for what seems to be an epilepsy issue.",
                        "The patient is struggling with opioid use, has undergone detox previously, and is interested in long-term rehab. They are seeking help for what seems to be an addiction psychiatry issue.",
                        "The patient reports tunnel vision, high eye pressure, and has been using prescribed eye drops. They are seeking help for what seems to be a glaucoma issue.",
                        "The patient presents with high creatinine levels, fluid retention, and signs of nephron loss. They are seeking help for what seems to be a chronic kidney disease issue."]

correct_subspecialties = ["Interventional Cardiology",
                          "Pediatric Dermatology",
                          "Stroke (Neurology)",
                          "Rheumatoid Arthritis",
                          "Pediatric Urology",
                          "Gynecologic Oncology",
                          "Epilepsy",
                          "Addiction Psychiatry",
                          "Glaucoma",
                          "Chronic Kidney Disease"]



def get_keywords(description):
    description = description.translate(str.maketrans('', '', string.punctuation))
    description = description.lower()
    description_split = description.split(' ')
    keywords  = [word for word in description_split if word not in stopwords]
    return keywords

keywords=get_keywords(patient_descriptions[0])

# First, you're going to need to import wordnet:
from nltk.corpus import wordnet

# Then, we're going to use the term "program" to find synsets like so:
syns = wordnet.synsets("program")

# An example of a synset:
print(syns[0].name())

# Just the word:
print(syns[0].lemmas()[2].name())





plan.n.01
programme


I can just use similarity scores instead of synonyms

In [19]:
#hit = wordnet.synset('heart_attack')
slap = wordnet.synset('interventional_cardiology')
hit.path_similarity(slap)

ValueError: not enough values to unpack (expected 3, got 1)

In [26]:
from nltk.corpus import wordnet as wn

word = 'ivf'
synsets = wn.synsets(word)

for synset in synsets:
    print(f"Synset: {synset.name()}")
    print(f"Definition: {synset.definition()}")
    print(f"Examples: {synset.examples()}")
    print("---")